<a href="https://colab.research.google.com/github/KTH-EXPECA/examples/blob/main/sdr/setup_sdr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Authentication and Dependencies

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [1]:
import os, re
from getpass import getpass

with open('/content/demo_project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

enter your expeca password:··········


Install required packages and dependencies. Ignore the warnings.

In [2]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 794.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.3/324.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.1/256.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.7/205.7 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.7/253.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.5/515.5 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

Import packages

In [3]:
import json, time
from loguru import logger
import chi.network, chi.container, chi.network
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed, show_reservation_byname, restart_sdr, make_sdr_ni, make_sdr_mango, sdr_tools, get_available_publicips, get_segment_ids, get_radio_interfaces, get_worker_interfaces

# Reserve resources

In the next cell, we reserve 1 SDR and 1 worker.

In [22]:
# Check the SDR's health and the status of its ports, both ports are supposed to be up, otherwise contact support
sdr_name = "sdr-10"
sdr_status = get_radio_interfaces(sdr_name)
logger.info(f"{json.dumps(sdr_status, indent=4)}")
for port in sdr_status.keys():
  if sdr_status[port]['linkstate'] == 'Down':
    logger.warning(f"port {port} on {sdr_name} is down.")
  if sdr_status[port]['linkstate'] == 'Up':
    logger.success(f"port {port} on {sdr_name} is up.")

2024-07-15 19:06:14.975 | INFO     | __main__:<cell line: 4>:4 - {
    "sdr_10_mango": {
        "backpressure": "Disabled",
        "duplex": "Full",
        "flowctrl": "Off",
        "linkstate": "Up",
        "mdixmode": "On",
        "neg": "Enabled",
        "port": "te1/0/2",
        "segment_id": 119,
        "speed": "1000",
        "stitches": {},
        "type": "10G-Copper"
    },
    "sdr_10_ni": {
        "backpressure": "Disabled",
        "duplex": "Full",
        "flowctrl": "Off",
        "linkstate": "Up",
        "mdixmode": "Off",
        "neg": "Disabled",
        "port": "te2/0/16",
        "segment_id": 120,
        "speed": "10000",
        "stitches": {},
        "type": "10G-Fiber"
    }
}
2024-07-15 19:06:14.977 | SUCCESS  | __main__:<cell line: 5>:9 - port sdr_10_mango on sdr-10 is up.
2024-07-15 19:06:14.978 | SUCCESS  | __main__:<cell line: 5>:9 - port sdr_10_ni on sdr-10 is up.


In [23]:
# Reserve the SDR
segment_ids = get_segment_ids(sdr_name)

# reserve RJ45 port
rj45_lease = show_reservation_byname(sdr_name + "-rj45-lease")
if not rj45_lease:
    rj45_lease = reserve(
        { "type":"network", "name": sdr_name+"-rj45", "net_name": sdr_name+"-rj45", "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )

# reserve SFP port
sfp_lease = show_reservation_byname(sdr_name + "-sfp-lease")
if not sfp_lease:
    sfp_lease = reserve(
        { "type":"network", "name": sdr_name+"-sfp", "net_name": sdr_name+"-sfp", "segment_id": segment_ids['sfp'], "duration": { "days":7, "hours":0 } }
    )

worker_name = 'worker-01'
# reserve worker
worker_lease = show_reservation_byname(worker_name+"-lease")
if not worker_lease:
    worker_lease = reserve(
        { "type":"device", "name":worker_name, "duration": { "days":7, "hours":0 } }
    )
worker_reservation_id = worker_lease["reservations"][0]["id"]


leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

2024-07-15 19:07:30.814 | INFO     | chi.expeca:reserve:243 - reserving sdr-10-rj45
2024-07-15 19:07:33.254 | INFO     | chi.expeca:wait_until_lease_status:138 - waiting 120 seconds for sdr-10-rj45-lease with id 18b7fbc7-1482-4e2f-9f13-632ee4500e1e to become "ACTIVE"
2024-07-15 19:07:38.422 | INFO     | chi.expeca:wait_until_lease_status:145 - lease sdr-10-rj45-lease with id 18b7fbc7-1482-4e2f-9f13-632ee4500e1e is PENDING.
2024-07-15 19:07:43.596 | INFO     | chi.expeca:wait_until_lease_status:145 - lease sdr-10-rj45-lease with id 18b7fbc7-1482-4e2f-9f13-632ee4500e1e is PENDING.
2024-07-15 19:07:48.759 | INFO     | chi.expeca:wait_until_lease_status:145 - lease sdr-10-rj45-lease with id 18b7fbc7-1482-4e2f-9f13-632ee4500e1e is PENDING.
2024-07-15 19:07:53.938 | INFO     | chi.expeca:wait_until_lease_status:145 - lease sdr-10-rj45-lease with id 18b7fbc7-1482-4e2f-9f13-632ee4500e1e is PENDING.
2024-07-15 19:07:59.103 | INFO     | chi.expeca:wait_until_lease_status:145 - lease sdr-10-rj45-

[
    {
        "name": "sdr-10-rj45-lease",
        "id": "18b7fbc7-1482-4e2f-9f13-632ee4500e1e",
        "reservation_id": "cc53d8fb-9b3e-4e65-9778-8b53dd6ab6a1",
        "status": "ACTIVE",
        "end_date": "2024-07-22T19:07:00.000000"
    },
    {
        "name": "sdr-10-sfp-lease",
        "id": "ad69a46f-013b-4f69-a486-a2ad3dbe06aa",
        "reservation_id": "ed41686e-061d-4fc3-b754-987d3452ced3",
        "status": "ACTIVE",
        "end_date": "2024-07-22T19:08:00.000000"
    },
    {
        "name": "worker-01-lease",
        "id": "ede1d246-2245-455a-a511-6623f79b204a",
        "reservation_id": "f17f42f8-2928-4df7-b922-d0b9e3ac0933",
        "status": "ACTIVE",
        "end_date": "2024-07-22T19:09:00.000000"
    }
]


# Configure the SDRs

## Change the sdrs' firmwares to Mango or NI

In [ ]:
# change sdr design to ni using the reserved worker
sdrnet = chi.network.get_network(sdr_name+"-rj45-net")
make_sdr_ni(sdr_name,sdrnet['id'],worker_reservation_id,"ens1")

# wait 10 seconds
#time.sleep(10)

# change sdr design to mango using the reserved worker
#sdrnet = chi.network.get_network(sdr_name+"-rj45-net")
#make_sdr_mango(sdr_name,sdrnet['id'],worker_reservation_id,"ens1")

2023-09-01 08:04:04.639 | SUCCESS  | chi.expeca:make_sdr_mango:299 - created make-sdr-mango container.
2023-09-01 08:04:04.641 | INFO     | chi.expeca:make_sdr_mango:301 - waiting 2 minutes for the sdr-09 to change design.
2023-09-01 08:05:06.410 | SUCCESS  | chi.expeca:make_sdr_mango:315 - SERVICE=change_design env variable is set
running change_design...
You have chosen sdr-09 and design mango
sdr-09 is reachable: {'mango': {'ip': '10.30.1.17', 'port': '22', 'tenant-port': 'te1/0/26', 'up': False}, 'ni': {'ip': '10.30.1.18', 'port': '22', 'tenant-port': 'te4/0/13', 'up': True}}
SSHing to 10.30.1.18, username:root, password: 
running command on 10.30.1.18: 
cp /uboot/mango_bootbin/BOOT.bin /uboot/ ; /sbin/reboot > /dev/null 2>&1 &
command stdout: 

command sdterr: 

Waiting 200 seconds for the new desgin to load
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
sdr-09 design has be

## Reboot the SDRs

In [ ]:
# reset sdr using the reserved worker
sdrnet = chi.network.get_network(sdr_name+"-rj45-net")
restart_sdr(sdr_name,sdrnet['id'],worker_reservation_id,"ens1")


2023-09-01 10:10:24.442 | SUCCESS  | chi.expeca:restart_sdr:255 - created reboot-sdr container.
2023-09-01 10:10:24.445 | INFO     | chi.expeca:restart_sdr:257 - waiting 2 minutes for the sdr-09 to reboot.
2023-09-01 10:11:15.129 | SUCCESS  | chi.expeca:restart_sdr:267 - SERVICE=reboot env variable is set
running reboot...
You have chosen sdr-09
sdr-09 is reachable: {'mango': {'ip': '10.30.1.17', 'port': '22', 'tenant-port': 'te1/0/26', 'up': True}, 'ni': {'ip': '10.30.1.18', 'port': '22', 'tenant-port': 'te4/0/13'}}
SSHing to 10.30.1.17, username:root, password: root
running command on 10.30.1.17: 
/sbin/reboot > /dev/null 2>&1 &
Waiting 200 seconds for the sdr to load
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
sdr-09 mango is up again.

2023-09-01 10:11:15.612 | INFO     | chi.expeca:wait_until_container_removed:30 - waiting 30 seconds for reboot-sdr container to be removed
2023-09-01 10:11:20.836 | INFO

# Run A Public Container to Test the SDR

Run a public container to test the SDR via its SFP port

In [27]:
# check public IPs and select one
available_pub_ips = get_available_publicips()
if len(available_pub_ips) == 0:
  raise Exception("There is no available public IPs to reserve.")
pub_ip = available_pub_ips[0]
logger.info(f"Available public ips: {available_pub_ips}.")
logger.info(f"We choose {pub_ip} for this container.")

# check available interfaces on the worker
interfaces = list(get_worker_interfaces(worker_name).values())[0]
available_ifs = []
for interface in interfaces.keys():
  if len(interfaces[interface]['connections']) == 0:
    available_ifs.append(interface)
logger.info(f"Available interfaces on {worker_name}: {available_ifs}")
# we need one 10Gbps interface for the SDR,
# and one port with any speed for the public interface
port_10g = None
port_any = None
for interface in available_ifs:
  if (port_10g is None) and (interfaces[interface]['speed'] == '10000'):
    port_10g = interface
    continue
  if port_any is None:
    port_any = interface
if (port_10g is None) or (port_any is None):
  logger.info(f"{json.dumps(interfaces, indent=4)}")
  raise Exception(f"Did not find proper interfaces on {worker_name}")
else:
  logger.success(f"we choose {port_10g} for the SDR and {port_any} for the public access")

# run the container
sdrsfpnet = chi.network.get_network(sdr_name+"-sfp-net")
publicnet = chi.network.get_network("serverpublic")
container_name = "my-public-sdr-host"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/sshd-image",
    reservation_id = worker_reservation_id,
    environment = {
        "DNS_IP":"8.8.8.8",
        "GATEWAY_IP":"130.237.11.97",
        "PASS":"expeca"
    },
    mounts = [],
    nets = [
        { "network" : publicnet['id'] },
        { "network" : sdrsfpnet['id'] },
    ],
    labels = {
        "networks.1.interface":available_ifs[0],
        "networks.1.ip":pub_ip+"/27",
        "networks.1.gateway":"130.237.11.97",
        "capabilities.privileged":"true",
        "networks.2.interface":available_ifs[1],
        "networks.2.ip":"10.30.10.120/24",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container, reachable at {pub_ip}.")


2024-07-15 19:18:27.017 | INFO     | __main__:<cell line: 6>:6 - Available public ips: ['130.237.11.114', '130.237.11.115', '130.237.11.116', '130.237.11.117', '130.237.11.119', '130.237.11.120', '130.237.11.121', '130.237.11.122', '130.237.11.123', '130.237.11.124', '130.237.11.125'].
2024-07-15 19:18:27.019 | INFO     | __main__:<cell line: 7>:7 - We choose 130.237.11.114 for this container.
2024-07-15 19:18:39.317 | INFO     | __main__:<cell line: 15>:15 - Available interfaces on worker-01: ['eno12399', 'eno12409', 'eno12419', 'eno12429', 'ens1f1']
2024-07-15 19:18:39.319 | SUCCESS  | __main__:<cell line: 26>:30 - we choose ens1f1 for the SDR and eno12399 for the public access
2024-07-15 19:19:11.573 | SUCCESS  | __main__:<cell line: 60>:60 - created my-public-sdr-host container, reachable at 130.237.11.114.
